In [40]:
from bs4 import BeautifulSoup
import requests
import json
import re
import pandas as pd
import math
from unidecode import unidecode

In [41]:
def pegar_html(link):  
    req=requests.get(link)
    html=BeautifulSoup(req.text,"html.parser")
    return html
url='https://www.drogariaveracruz.com.br/medicamentos/'


def num_pag(html):
    string=html.find_all('div',{'class':'text-center pt-3'})[1].text
    lista=string.split()
    paginas= int(lista[-1])
    return paginas


def get_links_produtos(html):
    div=html.find_all('div',{'class':'item-product'})
    LINKS=[]
    for caixa  in div:
        link_tag=caixa.find('a',{'class':'item-image position-relative'})
        link=link_tag.get('href')
        link="https://www.drogariaveracruz.com.br/"+link
        LINKS.append(link)
    return LINKS

def get_ean(html):
    div=html.find('div',{'class':'d-flex flex-wrap text-muted mb-3'})
    tag_gtin=div.find('meta',{'itemprop':'gtin13'})
    if tag_gtin == None:
        gtin="SEM EAN"
    else:
        gtin=tag_gtin.get('content')
        gtin=int(gtin)
    return gtin

def get_marca(html):
    div=html.find('div',{'class':'d-flex flex-wrap text-muted mb-3'})
    tag_marca=div.find('meta',{'itemprop':'brand'})
    if tag_marca == None:
        return "SEM MARCA REGISTRADA"
    else:
        marca=tag_marca.get('content')
        return marca


def get_preço_cheio(html):
    div=html.find('div',{'class':'position-relative'})
    if div.find('h2') != None:
        return "PRODUTO ESGOTADO"
    elif div.find('p',{'class':'unit-price'})==None:
        cheio = div.find('p',{'class':'sale-price'})
        
    else:
        cheio=div.find('p',{'class':'unit-price'}).text
        cheio=cheio.replace('\n                                                    R$ ','')
        cheio=cheio.replace('\n','')
        cheio=cheio.replace('.','')
        cheio=float(cheio.replace(',','.'))
        return cheio

def get_preço_venda(html):
    div=html.find('div',{'class':'position-relative'})
    if div.find('h2',) != None:
        return "PRODUTO ESGOTADO"
    else:
        venda=div.find('p',{'class':'sale-price'}).text
        venda=venda.replace('R$ ','')
        venda=venda.replace('\n','')
        venda=venda.replace('.','')
        venda=float(venda.replace(',','.'))
        return venda

def get_nome(html):
    nome=html.find('h1',{'class':'name','itemprop':'name'}).text
    return nome

def get_desconto(cheio,venda):
    if cheio == "PRODUTO ESGOTADO":
        return cheio
    else:
        dis=cheio-venda
        if dis<0:
            dis=dis*(-1)
        return dis




















In [43]:
url='https://www.drogariaveracruz.com.br/medicamentos/'
html=pegar_html(url)

numero_paginas=num_pag(html)


E=[]
M=[]
PC=[]
PV=[]
N=[]
D=[]
i=1
for pagina in range (numero_paginas):

    url=f'https://www.drogariaveracruz.com.br/medicamentos/?p={pagina}'
    html=pegar_html(url)
    LINKS_PRODUTOS=get_links_produtos(html)

    for url in LINKS_PRODUTOS:
        produto=pegar_html(url)
        ean=get_ean(produto)
        marca=get_marca(produto)
        preço_cheio=get_preço_cheio(produto)
        preço_venda=get_preço_venda(produto)
        nome=get_nome(produto)
        desconto=get_desconto(preço_cheio,preço_venda)

        E.append(ean)
        M.append(marca)
        PC.append(preço_cheio)
        PV.append(preço_venda)
        N.append(nome)
        D.append(desconto)
    print(f'{i}/153 paginas lidas')
    i+=1
df=pd.DataFrame({
    "EAN":E,
    "Marca":M,
    "Nome":N,
    "Preço sem desconto":PC,
    "Preço com desconto":PV,
    "Desconto":D

})  
df

1/153 paginas lidas
2/153 paginas lidas
3/153 paginas lidas
4/153 paginas lidas
5/153 paginas lidas
6/153 paginas lidas
7/153 paginas lidas
8/153 paginas lidas
9/153 paginas lidas
10/153 paginas lidas
11/153 paginas lidas
12/153 paginas lidas
13/153 paginas lidas
14/153 paginas lidas
15/153 paginas lidas
16/153 paginas lidas
17/153 paginas lidas
18/153 paginas lidas
19/153 paginas lidas
20/153 paginas lidas
21/153 paginas lidas
22/153 paginas lidas
23/153 paginas lidas
24/153 paginas lidas
25/153 paginas lidas
26/153 paginas lidas
27/153 paginas lidas
28/153 paginas lidas
29/153 paginas lidas
30/153 paginas lidas
31/153 paginas lidas
32/153 paginas lidas
33/153 paginas lidas
34/153 paginas lidas
35/153 paginas lidas
36/153 paginas lidas
37/153 paginas lidas
38/153 paginas lidas
39/153 paginas lidas
40/153 paginas lidas
41/153 paginas lidas
42/153 paginas lidas
43/153 paginas lidas
44/153 paginas lidas
45/153 paginas lidas
46/153 paginas lidas
47/153 paginas lidas
48/153 paginas lidas
4

,EAN,Marca,Nome,Preço sem desconto,Preço com desconto,Desconto
0,7896637032209,Apsen,Suplemento Alimentar Probid 30 Cápsulas,86.4,75.17,11.23
1,7891045031257,Nestlé,Suplemento Vitamínico Nestlé Materna 30 compri...,71.9,66.15,5.75
2,7896637032247,Apsen,Lactosil Flora 30 cápsulas,60.9,53.59,7.31
3,7891721201806,Merck,Glifage XR 500mg 30 comprimidos de liberação p...,11.65,7.46,4.19
4,7898928257032,Gsk,Niquitin 21mg 7 adesivos transdérmicos transpa...,125.49,125.49,0.0
...,...,...,...,...,...,...
3095,7896006201007,União Química,Urovit 200mg 18 drágeas,PRODUTO ESGOTADO,PRODUTO ESGOTADO,PRODUTO ESGOTADO
3096,7896112470311,Biolab Sanus,"Vasopril Plus 20mg + 12,5mg 60 comprimidos",PRODUTO ESGOTADO,PRODUTO ESGOTADO,PRODUTO ESGOTADO
3097,7896255766135,Abbott,"Venocur Fit 263,2mg 60 comprimidos revestidos ...",PRODUTO ESGOTADO,PRODUTO ESGOTADO,PRODUTO ESGOTADO
3098,7891317487317,Eurofarma,"Versa 40mg/0,4ml Solução Injetável 2 seringas ...",PRODUTO ESGOTADO,PRODUTO ESGOTADO,PRODUTO ESGOTADO


In [45]:
df.to_excel("VERA_CRUZ.xlsx")